In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
os.chdir('C:/My Documents/Mokymai ir kt info/Python/10_Final_Project')
os.getcwd()

# Main GIS Table

In [ ]:
read_gis_file = pd.read_excel('GIS_DB/CS_GIS_DATA.xlsx')
read_gis_file.to_csv('GIS_DB/CS_GIS_DATA.csv', index = None, header = True)
df_gis = pd.read_csv('GIS_DB/CS_GIS_DATA.csv', parse_dates=['Start', 'Finish'])

In [ ]:
df_gis.info()

In [ ]:
df_gis.head(2)

# Clean Weekly Data update table

In [ ]:
read_gis_file_update = pd.read_excel('Input/CS_Update_01_11_2021.xlsx', sheet_name= 1)
read_gis_file_update.to_csv('Input/CS_Update_01_11_2021.csv', index = False, header = True)
df_gis_update = pd.read_csv('Input/CS_Update_01_11_2021.csv')

In [ ]:
df_gis_update.info()

In [ ]:
df_gis_update.head(2)

In [ ]:
df_gis_update['Start'] = df_gis_update['Start'].str[:9]
df_gis_update['Finish'] = df_gis_update['Finish'].str[:9]
df_gis_update['At Completion Duration'] = df_gis_update['At Completion Duration'].str.replace('d','')

In [ ]:
df_gis_update.head(2)

In [ ]:
df_gis_update.to_csv('Input/CS_Update_01_11_2021.csv', index = False)

In [ ]:
df_gis_update = pd.read_csv('Input/CS_Update_01_11_2021.csv', parse_dates=['Start', 'Finish'])
df_gis_update.head(2)

In [ ]:
# df_gis_update.drop('Unnamed: 0', axis='columns', inplace=True)

In [ ]:
df_gis_update.info()

# Compare CSV's

In [ ]:
df_gis_update['act_cs_id'] = (df_gis_update['Activity ID'] +"/"+ df_gis_update['CS ID'])
df_gis['act_cs_id'] = (df_gis['Activity ID'] +"/"+ df_gis['CS ID'])

In [ ]:
df_gis_update.head(2)

In [ ]:
df_gis.head(2)

In [ ]:
def patikra():
    updvsorg = df_gis_update[~df_gis_update.act_cs_id.isin(df_gis.act_cs_id)]
    orgvsupd = df_gis[~df_gis.act_cs_id.isin(df_gis_update.act_cs_id)]
    if df_gis_update.act_cs_id.nunique() == len(df_gis_update.act_cs_id):
        if updvsorg.empty and orgvsupd.empty:
            return f'ID codes, number of object match. Total number of objects - {df_gis_update.act_cs_id.nunique()}. \
Data can be updated'
        elif len(updvsorg) > 0 and orgvsupd.empty:
            raise NotImplementedError (f"New objects have appeared ({len(updvsorg)} obj.), that are not in the GIS DB. \n{updvsorg}")
        elif len(orgvsupd) > 0 and updvsorg.empty:
            raise NotImplementedError (f"Missing objects that are in the GIS DB ({len(orgvsupd)} obj.). \n{orgvsupd}")
        elif len(updvsorg) > 0 and len(orgvsupd) > 0:
            raise NotImplementedError (f"ID's don't match/New objects are found/Missing objects.\
                \n \n Objects that are not in GIS DB: \n {updvsorg} \n \n Objects that are not in the updated table: \n {orgvsupd}")
    else:
        raise NotImplementedError (f"Duplicate IDs are not allowed")
print (patikra())

# Update GIS DB

In [ ]:
df_gis.loc[[1, 2, 7, 8, 9, 10, 11, 12]]

In [ ]:
df_gis_update.loc[[1, 25, 7, 8, 9, 10, 11, 12]]

In [ ]:
updated = df_gis.merge(df_gis_update, how='left', on=['act_cs_id'], suffixes=('', '_new'))
updated[['Start', 'Finish', 'At Completion Duration']] = np.where(pd.notnull(updated[['Start_new', 'Finish_new', 'At Completion Duration_new']]), updated[['Start_new', 'Finish_new', 'At Completion Duration_new']], updated[['Start_new', 'Finish_new', 'At Completion Duration_new']])
updated.drop(['act_cs_id', 'Start_new', 'Finish_new', 'Activity ID_new', 'Activity Name_new', 'At Completion Duration_new', 'CS ID_new', 'Stages_new'], axis=1, inplace=True)

In [ ]:
updated.loc[[1, 2, 7, 8, 9, 10, 11, 12]]

In [ ]:
updated.to_csv('Output/updated.csv', index = False, header = True)
updated.to_excel('Output/updated.xlsx', index = False, header = True)

In [ ]:
read_new_excel = pd.read_excel('Output/updated.xlsx', sheet_name= 0)

read_new_excel.info()

In [ ]:
read_new_excel